# Setup

```python
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
```

In [ ]:
import os
from pathlib import Path
from google.colab import drive

# drive.mount("/content/gdrive")
# ROOT = Path("/content/gdrive/My Drive/Colab Notebooks")
# BASE = ROOT.joinpath('LOads')
BASE = Path("/content/gdrive/My Drive/Notebooks/LOads_colab")
BASE.mkdir(exist_ok=True)
os.chdir(BASE)


In [ ]:
!apt-get install libspatialindex-dev python3-rtree
!pip install geopandas
!ldconfig
from rtree import index
from rtree.index import Rtree
import numpy as np
import pandas as pd
import geopandas
from collections import namedtuple
from dateutil.parser import parse
from datetime import timedelta
import json
from matplotlib import pyplot as plt

In [ ]:
def merge(left, right, on, cols):
    for col in cols:
        if col in left.columns:
            left = left.drop(columns=col)
    right = right.reset_index()
    return left.merge(right[on+cols], on=on, how='left')

In [ ]:
# uploaded get_wqdata.py to Colab Notebooks folder, now add to import path
import sys
if BASE not in sys.path:
    sys.path.append(str(BASE))
from get_wqdata import GetWQData, WQ_API_URL

import get_wqdata
from importlib import reload
reload(get_wqdata)
GetWQData = get_wqdata.GetWQData
WQ_API_URL = get_wqdata.WQ_API_URL

wq = GetWQData()

# Explore dataset

Want to work out which CharacteristicName values to request.  The WQ portal web interface says they come [from here](http://iaspub.epa.gov/sor_internet/registry/substreg/home/overview/home.do) but that's hard to search, so request all obs. in our bounding box and search that list instead.

In [ ]:
query = dict(
    _desc="All sample types",
    bBox="-80.0402,43.0646,-75.9697,44.355",
    siteType=["Aggregate surface-water-use", "Stream"],
    sampleMedia=["Water", "water"],
    # siteid=["USGS-04231600"],
    # startDateLo="01-01-2018",
    # startDateHi="02-01-2018",
)
print("Reading data")
all_data = wq.get_data(query)  # gets path to .csv file
d = pd.read_csv(all_data, low_memory=False)  # , nrows=10)
print("Data read")
print(d.columns)
po4 = set(d["CharacteristicName"])
keep = set()
for term in 'po4', 'phosphor', 'srp', 'phosphate', 'flow', 'guage', 'cfs':
    for cname in po4:
        if term in cname.lower():
            keep.add(cname)
print(sorted(keep))

Now make a sensible list from the values we found.

In [ ]:
use_cache = {
    'characteric_names': [
        # 'O-Ethyl O-methyl S-propyl phosphorothioate',
        # 'O-Ethyl S-methyl S-propyl phosphorodithioate',
        # 'O-Ethyl S-propyl phosphorothioate',
        # 'Organic phosphorus',
        'Orthophosphate',
        'Orthophosphate as P',
        'Phosphate-phosphorus',
        'Phosphate-phosphorus as P',
        'Phosphorus',
        'Stream flow, instantaneous',
        'Stream flow, mean. daily',
        # 'Tributyl phosphate',
        # 'Triphenyl phosphate',
        # 'Tris(1,3-dichloro-2-propyl)phosphate',
        # 'Tris(2-butoxyethyl) phosphate',
        # 'Tris(2-chloroethyl) phosphate',
    ]
}

names = use_cache.get('characteric_names')

Now grab all the data we're interested in

In [ ]:
query = dict(
    _desc="Target sample types",
    bBox="-80.0402,43.0646,-75.9697,44.355",
    siteType=["Aggregate surface-water-use", "Stream"],
    sampleMedia=["Water", "water"],
    characteristicName=names,
)
print("Reading data")
# first get the Site info. flavored response
data = wq.get_data(query, type_='site')  # gets path to .csv file
site = pd.read_csv(data)
# print('\n'.join(sorted(site.columns)))
# then get the Result info.
data = wq.get_data(query, type_='result')
d = pd.read_csv(data, low_memory=False)
d.columns = [i.replace('/', '_') for i in d.columns]
# get rid of "mg/l<space><space><space>" units
for col in ['ResultMeasure_MeasureUnitCode']:
    d[col] = [str(i).strip() for i in d[col]]
had = len(d)
d = d.loc[d['ResultMeasure_MeasureUnitCode'] != 'nan', :]
print("Lost %d for missing units" % (had - len(d)))


In [ ]:
# find common columns
common = set(site.columns).intersection(set(d.columns))
print('\n'.join(sorted(common)))
# use this common column
common = 'MonitoringLocationIdentifier'

# Merge sites

In [ ]:
# create a list of unique sites
locs = site.loc[:, [common, 'LatitudeMeasure', 'LongitudeMeasure']]
locs.drop_duplicates(inplace=True)
length = len(locs)
length

In [ ]:
# copy lat / lon to main data
old_len = len(d)
locs.reset_index(inplace=True, drop=True)
d = d.merge(locs.loc[:, [common, 'LatitudeMeasure', 'LongitudeMeasure']], on=[common])
assert len(d) == old_len
assert sum(d.LatitudeMeasure == 0) == 0
assert sum(np.isnan(d.LatitudeMeasure)) == 0

Data has different `MonitoringLocationIdentifier`s for the same coords.

In [ ]:
# generate a location based site field
lonlat = d[['LongitudeMeasure', 'LatitudeMeasure']].drop_duplicates()
dd = geopandas.GeoDataFrame(
    lonlat,
    geometry=geopandas.points_from_xy(
        lonlat.LongitudeMeasure, lonlat.LatitudeMeasure
    ),
    crs = {'init' :'epsg:4326'},
)
dd['site'] = dd.LatitudeMeasure.astype(str)+dd.LongitudeMeasure.astype(str)
dd.drop(columns=['LongitudeMeasure', 'LatitudeMeasure'], inplace=True)
# 50 m buffer merges sites within 100 m of each other
dd.geometry = dd.geometry.to_crs(5071).buffer(50)
intersect = geopandas.overlay(dd, dd, how='identity')
lint = len(intersect)
intersect = intersect[intersect.site_2.astype(str) != 'nan']
print("Dropped %d NaN intersections" % (lint-len(intersect)))

# make a dict of site: (set of intersecting sites)
sets = {}
for inter in intersect.itertuples():
    site_1 = str(inter.site_1)
    site_2 = str(inter.site_2)
    sets.setdefault(site_1, set()).add(site_2)
    sets.setdefault(site_2, set()).add(site_1)
# pick the lowest (alphabetically) site as the name for intersecting sites
sites = {min(i):i for i in sets.values()}
# sites = {(k+"_X" if len(v) > 1 else k):v for k, v in sites.items()}
[{k:v} for k,v in sites.items() if len(v) > 1]
len(lonlat), len(sites)
site = {}
xtra = {}
for k, v in sites.items():
    for s in v:
        site[s] = k
        xtra[s] = len(v) > 1

lu = d.loc[:, ['LongitudeMeasure', 'LatitudeMeasure', common]].drop_duplicates(
    subset=['LongitudeMeasure', 'LatitudeMeasure'])
#X lu = merge(lu, d, ['LongitudeMeasure', 'LatitudeMeasure'], [common])
lu['sitehash'] = lu.LatitudeMeasure.astype(str)+lu.LongitudeMeasure.astype(str)
sitename = {k:v for k,v in zip(lu.sitehash, lu[common])}
lu['site'] = [sitename[site[i]]+('_X' if xtra[i] else '') for i in lu.sitehash]
lu.drop_duplicates(inplace=True)

d = merge(d, lu, ['LongitudeMeasure', 'LatitudeMeasure'], ['site'])
assert len(d) == old_len, (old_len, len(d))   

Now copy site info. into locs table

In [ ]:
nlocs = merge(locs, d, [common, 'LatitudeMeasure', 'LongitudeMeasure'], ['site'])
locs_len = len(nlocs)
nlocs.drop_duplicates(inplace=True, subset=['site', 'MonitoringLocationIdentifier'])
assert len(nlocs) == length, (length, len(nlocs))
locs = nlocs
locs.to_csv("locs.csv", index=False)

In [ ]:
for na in '', 'nan', 'NaN':
    assert sum(locs['site'] == na) == 0, na
    assert sum(d['site'] == na) == 0, na
    assert sum(locs['site'] == na+na) == 0, na
    assert sum(d['site'] == na+na) == 0, na

In [ ]:
print("%d locations" % len(locs))
distinct = locs.set_index(["LatitudeMeasure", "LongitudeMeasure"])
print("%d distinct" % len(distinct.index.unique()))

# Data to sampling events

In [ ]:
# tabulate units used for results
d['ResultSampleFractionText'] = [
    i if i != 'nan' else 'NA'
    for i in d['ResultSampleFractionText'].astype(str)
]

res_types = (
    d.groupby(
        [
            'CharacteristicName',
            'ResultMeasure_MeasureUnitCode',
            'ResultSampleFractionText',
        ]
    )
    .count()
    .iloc[:, 0]
)
print(res_types)

In [ ]:
duration = wq.db.get('duration', [])  # cache expensive calc. in wq.db
if not duration:
    for row in d.itertuples():
        try:
            t0 = parse(
                "%s %s"
                % (row.ActivityStartDate, row.ActivityStartTime_Time)
            )
            t1 = parse(
                "%s %s" % (row.ActivityEndDate, row.ActivityEndTime_Time)
            )
            duration.append((t1 - t0).seconds)
        except ValueError:
            duration.append(0)
    wq.db['duration'] = duration
d['duration'] = duration

Create an ID field for each sampling event

In [ ]:
d['sampling'] = (
    d['ActivityStartTime_Time'].astype(str)
    + d['ActivityEndTime_Time'].astype(str)
    + ' '
    + d['ActivityStartDate'].astype(str)
    + ' '
    + d['ActivityEndDate'].astype(str)
    + ' '
    + d['LatitudeMeasure'].astype(str)
    + ' '
    + d['LongitudeMeasure'].astype(str)
)

## Save output data

In [ ]:
d.to_csv('data.csv')

Now we want to match P conc. data and flow data for each sampling event

(start to refer to `d` as `data` from here on)

In [ ]:
data = pd.read_csv("data.csv", low_memory=False)
pdata = data.loc[:, ['sampling', 'ActivityStartDate', 'ActivityEndDate', 'site']]
for na in '', 'nan', 'NaN':
    assert sum(locs['site'] == na) == 0, na
    assert sum(d['site'] == na) == 0, na
    assert sum(pdata['site'] == na) == 0, na
    assert sum(locs['site'] == na+na) == 0, na
    assert sum(d['site'] == na+na) == 0, na
    assert sum(pdata['site'] == na+na) == 0, na
pdata.drop_duplicates(subset=['sampling'], inplace=True)
pdata_len = len(pdata)
print("%d into %d" % (len(data), pdata_len))
cname = 'CharacteristicName'
cunit = 'ResultMeasure_MeasureUnitCode'
vname = 'ResultMeasureValue'
TP = namedtuple("ToProcess", "to cname unit trans")
po4_to_p = (30.97 + 4 * 16) / 30.97
to_process = [
    TP('flow', 'Stream flow, instantaneous', 'ft3/s', None),
    TP(
        'flow',
        'Stream flow, instantaneous',
        'm3/sec',
        lambda x: x * 35.3147,
    ),
    TP('flow', 'Stream flow, mean. daily', 'ft3/s', None),
    TP('conc', 'Phosphorus', 'mg/l as P', None),
    TP('conc', 'Phosphorus', 'mg/l', None),
    TP('conc', 'Phosphorus', 'ug/l', lambda x: 1000 * x),
    TP('conc', 'Phosphorus', 'mg/l PO4', lambda x: x / po4_to_p),
    TP('conc', 'Phosphate-phosphorus as P', 'mg/l', None),
    TP('conc', 'Phosphate-phosphorus', 'mg/l', None),
    TP('conc', 'Orthophosphate as P', 'mg/l', None),
    TP('conc', 'Orthophosphate', 'mg/l as P', None),
    TP('conc', 'Orthophosphate', 'mg/l asPO4', lambda x: x / po4_to_p),
    TP('conc', 'Orthophosphate', 'mg/l', lambda x: x / po4_to_p),
    # exclude as it's in bed sediment
    # TP('conc', 'Phosphorus', 'mg/kg as P', None),
]

We're merging 82386 observations into 21877 flow + conc. records.  In a perfect world we'd have one flow and one conc. record for each sampling, but we have both redundant and mismatched (flow without conc. and visa versa) records, roughly 4:1 instead of 2:1.

`to_process` is an **ordered** list of items where each item reocords the field we're trying to fill (`to`), the `CharacteristicName` (`cname`) of our prefered source field, our prefered `unit`, and any conversion (`trans`) necessary to use this source field / unit combination.

Now we're going to fill the `to` fields (`flow`, `conc`) with the values selected using the list items, filling as many records with the first set of selected data, and only filling missing records with the next (less desireable) list item.

In [ ]:
selected = 0
data.set_index('sampling', drop=False, inplace=True)
pdata.set_index('sampling', drop=False, inplace=True)
for tp_i, tp in enumerate(to_process):
    select = np.logical_and(
        data[cname] == tp.cname, data[cunit] == tp.unit
    )
    selected += sum(select)
    print(
        "%d Selected %d %s %s (%d)"
        % (
            tp_i, sum(select),
            tp.cname,
            tp.unit,
            len(data['sampling'][select].unique()),
        )
    )
    mean = data.loc[select, :]
    mean = mean.groupby(level=0).mean()
    pdata = pdata.join(mean.loc[:, vname])
    assert len(pdata) == pdata_len, (pdata_len, len(pdata))
    if tp.trans is not None:
        pdata[vname] = pdata[vname].map(tp.trans)

    def missing(x, f):
        if f not in x.columns:
            return np.ones(len(x)).astype(bool)
        x = x[f].astype(str)
        ans = np.logical_or(np.equal(x, 'None'), np.equal(x, ''))
        ans = np.logical_or(ans, np.equal(x, 'nan'))
        ans = np.logical_or(ans, np.equal(x, 'NaN'))
        return ans

    missing0 = missing(pdata, tp.to)
    if tp.to not in pdata.columns:
        pdata.rename(columns={vname: tp.to}, inplace=True)
    else:
        # print(len(missing), sum(missing), missing.shape)
        #X pdata[tp.to][missing0] = pdata[vname][missing0]
        pdata.loc[missing0, tp.to] = pdata[vname][missing0]
        pdata.drop(columns=vname, inplace=True)
    missing1 = missing(pdata, tp.to)
    print(
        "%d, Needed %d, used %d, still need %d"
        % (tp_i, sum(missing0), sum(missing0) - sum(missing1), sum(missing1))
    )
    assert len(pdata) == pdata_len, (pdata_len, len(pdata))
print("Selected %s total" % selected)
pdata = pdata[~np.isnan(pdata['conc'])]
# pdata = pdata[~np.isnan(pdata['flow'])]
print("Lost %d to blank data" % (pdata_len-len(pdata)))
pdata.to_csv("pdata.csv")

Of 22,000 sampling events sampling either flow or P, we can't find flow *and* P for 4700.  Most (~3600) of the missing data is flow, so we *could* fill in with flow data, not doing that currently.

Update: not dropping obs. for missing flow data now.

In [ ]:
# update locs with the number of obs. for each site
count = pdata.groupby(by=['site']).count()
count.rename(columns={count.columns[0]:'count'}, inplace=True)
locs = merge(locs, count, ['site'], ['count'])

# as above for min/max date
count = pdata.groupby(by=['site']).min()
locs = merge(locs, count, ['site'], ['ActivityStartDate'])
locs.rename(columns={'ActivityStartDate': 'StartDate'}, inplace=True)
count = pdata.groupby(by=['site']).max()
# yes, use ActivityStartDate both times
locs = merge(locs, count, ['site'], ['ActivityStartDate'])
locs.rename(columns={'ActivityStartDate': 'EndDate'}, inplace=True)

locs.to_csv("locs.csv")

# Link to flow data

Now we need flow data for places without conc. data.  Doesn't seem to be in the WQ portal, not strictly speaking a WQ parameter (well, it's Water Quantity, not Water Quality).

Ran //waterservices.usgs.gov/nwis/site/?format=rdb&bBox=-80.040200,43.064600,-75.969700,44.355000&seriesCatalogOutput=true&siteType=ST&siteStatus=all and created `waterservices.usgs.gov.txt`


In [ ]:
floc = pd.read_csv(
    "waterservices.usgs.gov.txt", sep=r'\t', comment='#', engine='python', 
    skiprows=[43],  # the FORTRANesque column widths
    dtype={'site_no': str}
)
print("Total", len(floc))
floc.drop_duplicates(subset=['dec_lat_va', 'dec_long_va', 'agency_cd', 'site_no'], inplace=True)
print("Unique", len(floc))
floc = floc[~np.isnan(floc.dec_lat_va)]
floc = floc[~np.isnan(floc.dec_long_va)]
print("Non-null", len(floc))
floc.to_csv("locs2.csv")

The P sampling `locs` have their original lat/lon values, no necessarily consistent within a site.  That's ok, use these for intersect as it will give all possible answers for each site, we can resolve mulitples later.

In [ ]:
import folium
mdnmap = folium.Map(location=[43.234853, -77.533949], zoom_start=19)

for row in locs.itertuples():
    folium.Marker([row.LatitudeMeasure, row.LongitudeMeasure], popup=row.site).add_to(mdnmap)
mdnmap

# ---
locblobs = geopandas.GeoDataFrame(
    locs,
    geometry=geopandas.points_from_xy(
        locs.LongitudeMeasure, locs.LatitudeMeasure
    ),
    crs = {'init' :'epsg:4326'},
)
locblobs.geometry = locblobs.geometry.to_crs(5071).buffer(50)

# don't do this because we might want to use an up/down stream guage for
# an unguaged location with lots of P obs.
# although this doesn't entirely make sense as we're filtering on the presence
# of at least 1 WQ sample of some kind...
#
# print("From", len(locblobs), "locations")
# locblobs = locblobs[~np.isnan(locblobs['count'])]
# print("Go to", len(locblobs), "with P obs.")

flocblobs = geopandas.GeoDataFrame(
    floc,
    geometry=geopandas.points_from_xy(
        floc.dec_long_va, floc.dec_lat_va
    ),
    crs = {'init' :'epsg:4326'},
)
flocblobs.geometry = flocblobs.geometry.to_crs(5071).buffer(50)
# flocblobs['site_id'] = flocblobs.agency_cd + '-' + flocblobs.site_no
# because geopandas.overlay seems to change site_no to float

intersect = geopandas.overlay(locblobs, flocblobs, how='identity')

floc.site_no.dtype
flocblobs.site_no.dtype
intersect.site_no.dtype
# intersect.site_id.dtype


lint = len(intersect)
intersect = intersect[intersect.site_no.astype(str) != 'nan']
print("Dropped %d NaN intersections" % (lint-len(intersect)))

# make a dict of site: (set of intersecting flow data)
sets = {}
for inter in intersect.itertuples():
    site_1 = str(inter.site)
    site_2 = "%s-%s" % (inter.agency_cd, inter.site_no)
    sets.setdefault(site_1, set()).add(site_2)

[{k:v} for k,v in sets.items() if len(v) > 1]
toget = set()
for flows in sets.values():
    toget.update(flows)

In [ ]:
tsv_tmpl = {
    "site_no": 431510077363501,
    # "agency_cd": "USGS",
    "inventory_output": "0",
    "rdb_inventory_output": "file",
    "begin_date": "2000-01-01",
    "end_date": "2019-07-11",
    "TZoutput": "0",
    "pm_cd_compare": "Greater than",
    "radio_parm_cds": "all_parm_cds",
    "qw_attributes": "0",
    "format": "rdb",
    "qw_sample_wide": "wide",
    "rdb_qw_attributes": "0",
    "date_format": "YYYY-MM-DD",
    "rdb_compression": "gz",
    "submitted_form": "brief_list",
}

In [ ]:
# uploaded get_wqdata.py to Colab Notebooks folder, now add to import path
import sys
if BASE not in sys.path:
    sys.path.append(str(BASE))
from get_wqdata import GetWQData, WQ_API_URL

import get_wqdata
from importlib import reload
reload(get_wqdata)
GetWQData = get_wqdata.GetWQData
WQ_API_URL = get_wqdata.WQ_API_URL

wq = GetWQData()

In [ ]:
flows = {}    
params = 'p00060', 'p00061'
for flow in toget:
    flow = flow.replace("USGS-", "")
    # if flow != "04249000":
    #     continue
    query = dict(
        format="json",
        indent="off",
        sites=flow,
        startDT="2000-01-01",
        endDT="2019-07-10",
        parameterCd="00060",  # ft3/s
    )
    jsonfile = wq.get_data(query, 'flow')
    # print(WQ_API_URL['flow']+'?'+'&'.join("%s=%s" % (k, v) for k, v in query.items() if not k.startswith('_')))
    data = json.load(open(jsonfile))
    data = data['value']['timeSeries']
    if data:
        data = data[0]['values']
        assert len(data) == 1, len(data)
        data = data[0]['value']
        # print(flow, len(data))
        flows[flow] = [(i['dateTime'], float(i['value'])) for i in data]
        print("%s -> %s" % (flow, jsonfile))
    else:
        # fall back to TSV data
        query = tsv_tmpl.copy()
        query['site_no'] = str(flow)
        try:
            datapath = wq.get_data(query, 'flowTSV')
            print(datapath)
            data = pd.read_csv(datapath)     
            print("%s -> %s" % (flow, datapath))
            for param in params:
                if param not in data.columns:
                    print("No", param)
                    continue
                flows[flow] = [(row['sample_dt'], row[param]) for row_i, row in data.iterrows()]
        except OSError:  # not a gzip file
            print("%s -> %s" % (flow, 'NO DATA'))
            continue

# flows['431510077363501']

In [ ]:

locs['sFlow'] = None
locs['eFlow'] = None
locs['cFlow'] = None
for loc in locs.itertuples():
    if not loc.count:
        continue
    if loc.site not in sets:
        print(loc.site, "no flow data source found")
        continue
    flow = loc.site.replace("USGS-", "")
    if flow not in flows:
        print(loc.site, "no flow data found")
        continue
    print("Populating", loc.site)
    for src in sets[loc.site]:
        if loc.sFlow:
            print("Overwriting data for", loc.site)
        locs.loc[loc.Index, 'sFlow'] = min(i[0] for i in flows[flow])
        locs.loc[loc.Index, 'eFlow'] = max(i[0] for i in flows[flow])
        locs.loc[loc.Index, 'cFlow'] = len(flows[flow])
        # print(len(flows[flow]), 'for', locs.site)
locs.to_csv("locs.csv")
sets['USGS-431510077363501']
# flows['431510077363501']

## Generate LOADEST inputs

Needs two .csv files, `date, time, flow` for "estimation", and `date, time, conc, flow` for "calibration".

Pull in flows from flow data then interpolate flow data with interpolation distance (in days).  Interpolate three sources of flow data, the water quality portal data in `pdata`, the daily values in flows, and the TSV data, also in flows.

In [ ]:
def start_end_date(s, e):
    """Calculate midpoint time from start and end"""
    s = parse(s)
    try:
        e = parse(e)
    except AttributeError:  # when parse() gets a NaN from pandas
        e = s
    return s + (e - s) / 2  # midpoint from adding half timedelta

In [ ]:
pdata['date'] = [
    start_end_date(i.ActivityStartDate, i.ActivityEndDate).date()
    for i in pdata.itertuples()
]

In [ ]:
# first, collect actual flow observations by site
flow_itrp = {}
for row in locs.itertuples():
    site = row.site
    flow_obs = []
    for flow in pdata[pdata.site == site].itertuples():
        if np.isnan(flow.flow):
            continue
        s = flow.date
        flow_obs.append((s, flow.flow, 'pflow'))
    for flow in flows.get(site.replace("USGS-", ""), []):
        try:
            f = float(flow[1])
        except ValueError:
            continue  # "E 1.0" etc.
        if np.isnan(f):
            continue
        src = 'daily' if 'T' in flow[0] else 'TSV'
        flow_obs.append((parse(flow[0]).date(), f, src))
    if flow_obs:
        flow_obs.sort()
        flow_itrp[site] = flow_obs
# [i for i in flow_itrp['USGS-04232050'] if i[0].year == 2016 and i[0].month == 6]

In [ ]:
# then interpolate these actual observation lists
max_interp = 10  # days
for site in flow_itrp:
    flow_obs = flow_itrp[site]
    res = []
    i = 0
    while i < len(flow_obs) - 1:
        tsep = (flow_obs[i+1][0] - flow_obs[i][0]).days
        date = flow_obs[i][0]
        if tsep > max_interp:
            i += 1
            res.append((date, flow_obs[i][1], 0))
            continue
        vsep = flow_obs[i+1][1] - flow_obs[i][1]
        while date < flow_obs[i+1][0]:
            prop = (date - flow_obs[i][0]).days / tsep
            itrp = flow_obs[i][1] + prop * vsep
            dist = min(prop*tsep, (1-prop)*tsep)
            res.append((date, itrp, dist))
            date += timedelta(days=1)
        i += 1
        while i < len(flow_obs) - 2 and date == flow_obs[i+1][0]:
            i += 1
    flow_itrp[site] = res

    outpath = Path("loadests").joinpath(site)
    outpath.mkdir(parents=True, exist_ok=True)
    outfile = outpath.joinpath("%s_est.csv" % site)
    res = pd.DataFrame(data=dict(
        date=[i[0].strftime("%Y%m%d") for i in res],
        time='12:00',
        flow=[i[1] for i in res],
        itrp=[i[2] for i in res],
    ))
    res.to_csv(outfile, index=False)


In [ ]:
# sites where interpolation occured
' '.join([k for k, v in flow_itrp.items() if any(i[2] > 0 for i in v)])

In [ ]:
n = len(flow_itrp)
flow_itrp = {k:v for k, v in flow_itrp.items() if len(v) >= 20}
print("Dropped %d sites < 20 flow obs." % (n-len(flow_itrp)))

In [ ]:
ddropped = 0
fdropped = 0

for site in flow_itrp:
    if False and site != 'USGS-04232050':
        continue
        
    calobs = []
    sitep = pdata[pdata.site == site].copy()
    # sitep['date'] = [
    #     start_end_date(i.ActivityStartDate, i.ActivityEndDate)
    #     for i in sitep.itertuples()
    # ]
    n = len(sitep)
    sitep.drop_duplicates(subset=['date'], inplace=True)
    n -= len(sitep)
    if n:
        print("Dropped %s %d obs. on repeated dates" % (site, n))
        ddropped += n
    iflow = {i[0]:i[1] for i in flow_itrp.get(site, [])}
    sitep['iflow'] = [iflow.get(i, np.NaN) for i in sitep.date]
    # use general flow only in absence of specific flow from WQ data
    missing = np.isnan(sitep.flow)
    sitep.loc[missing, 'flow'] = sitep.loc[missing, 'iflow']
    n = len(sitep)
    sitep = sitep[~np.isnan(sitep.flow)]
    n -= len(sitep)
    if n:
        print("Dropped %s %d obs. for no flow data" % (site, n))
        fdropped += n
    outpath = Path("loadests").joinpath(site)
    outpath.mkdir(parents=True, exist_ok=True)
    outfile = outpath.joinpath("%s_calib.csv" % site)

    estfile = outpath.joinpath("%s_est.csv" % site)
    pltfile = outpath.joinpath("%s_data.pdf" % site)
    sitef = pd.read_csv(estfile)
    plt.clf()
    plt.gcf().set_size_inches(20, 6)
    # this just runs without returning
    # markerline, stemlines, baseline = plt.stem(sitep.date, sitep.flow)
    # plt.setp(stemlines, 'linewidth', 0.3)
    # markerline, stemlines, baseline = plt.stem([parse(str(i)) for i in sitef.date], sitef.flow)
    # plt.setp(stemlines, 'linewidth', 0.3)
    plt.plot([parse(str(i)) for i in sitef.date], sitef.flow, lw=0.3)  #, s=0.8)
    plt.scatter(sitep.date, sitep.flow, lw=0.3, s=0.4)
    ax2 = plt.gca().twinx()
    ax2.scatter(sitep.date, sitep.conc, s=0.5, c='k')
    plt.savefig(pltfile)

    sitep = pd.DataFrame(data=dict(
        date=[i.strftime("%Y%m%d") for i in sitep.date],
        time='12:00',
        flow=sitep.iflow,
        conc=sitep.conc,
    ))
    sitep.to_csv(outfile, index=False)
        
print("Dropped %d obs. on repeated dates" % ddropped)
print("Dropped %d P obs. with no flow" % fdropped)

In [ ]:
[i for i in flow_itrp['USGS-04232050'] if i[0].year == 2016 and i[0].month == 6]

In [ ]:
[i for i in flows['04232050'] if '2016-06-13' in i[0]]

In [ ]:
parse('2016-06-13T00:00:00.000')